In [4]:
# Pandas dataframe
import pandas as pd
from pandas import DataFrame

# Scikit-Learn
import sklearn 
from sklearn import datasets, linear_model, metrics, tree
from sklearn.model_selection import train_test_split, LeaveOneOut, KFold, cross_validate, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, accuracy_score

# Models
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso, Ridge
from sklearn.ensemble import (BaggingRegressor, ExtraTreesRegressor, GradientBoostingRegressor, 
RandomForestRegressor, AdaBoostRegressor)
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso,  RidgeCV
from sklearn.neighbors import KNeighborsRegressor
        
import joblib
from matplotlib import pyplot as plt
import numpy as np
import os
from tqdm import tqdm

In [5]:
df_size = pd.read_csv("dataset_scaled_size.csv")
df_size

,Unnamed: 0,metal_amount_mmol,ligand_amount_mmol,reductant_amount_mmol,sol1_vol_mL,solv2_vol_mL,time_min,temp_c,7,8,9,10,11,size_nm,g_factor*10^4.1
0,0,-0.454943,-0.187770,-0.196938,-0.123491,-0.525157,0.680663,-0.997007,1.0,46.0,15.0,9.0,1.0,3.06,0
1,1,-0.454943,-0.187770,-0.196938,-0.103038,-0.525157,0.680663,-0.997007,1.0,46.0,15.0,9.0,1.0,1.00,0.757989206
2,2,1.601355,0.032059,0.273730,-0.174625,-0.525157,-0.593509,-0.997007,6.0,71.0,15.0,9.0,1.0,1.00,0.58306862
3,3,1.768534,0.047047,0.285130,-0.031450,-0.525157,-0.593509,-0.997007,6.0,47.0,15.0,9.0,1.0,2.37,0
4,4,-0.454943,-0.187770,-0.196938,-0.123491,-0.525157,0.680663,-0.997007,1.0,71.0,15.0,9.0,1.0,4.00,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,192,-0.819253,-0.452414,-0.268488,-0.745892,-0.525157,-0.651426,-0.997007,6.0,37.0,15.0,9.0,1.0,20.00,0
140,197,0.570420,-0.202758,0.002838,-0.174625,-0.006407,-0.535592,-0.176375,6.0,79.0,15.0,9.0,4.0,1.05,15
141,198,0.570420,-0.202758,0.002838,-0.174625,-0.006407,-0.535592,-0.176375,6.0,79.0,15.0,9.0,4.0,1.30,10
142,199,0.570420,-0.327661,0.002838,-0.174625,-0.006407,-0.535592,-0.176375,6.0,79.0,15.0,9.0,4.0,1.05,7


In [6]:
input_col = [col for col in df_size.columns]
input_col.remove('Unnamed: 0')
input_col.remove('size_nm')
input_col.remove('g_factor*10^4.1')
input_col

['metal_amount_mmol',
 'ligand_amount_mmol',
 'reductant_amount_mmol',
 'sol1_vol_mL',
 'solv2_vol_mL',
 'time_min',
 'temp_c',
 '7',
 '8',
 '9',
 '10',
 '11']

In [7]:
output_col = ['size_nm']

X = df_size[input_col]

Y = df_size[output_col]

In [8]:
# Splitting dataset for training
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15, random_state=45, shuffle=True)

## Bagging

In [9]:
min_mae = 99999

min_i, min_j, min_k  = 0, 0, 0

for i in tqdm(range(1, 20)):
    for j in range(1, 20):
        for k in range(2, 36, 2):
            
            B_regr = BaggingRegressor(base_estimator=DecisionTreeRegressor(max_depth=i),
                                      n_estimators=j,
                                      random_state=k)
            
            B_regr.fit(X_train, np.ravel(Y_train))
            
            B_Y_pred = B_regr.predict(X_test)
            
            mae = mean_absolute_error(Y_test, B_Y_pred)
            
            if (min_mae > mae):
                min_mae = mae
                min_i = i
                min_j = j
                min_k = k
            
print(min_mae, min_i, min_j, min_k)

100%|██████████| 19/19 [02:23<00:00,  7.56s/it]

1.6617045454545454 11 1 26


## Random Forest

In [11]:
min_mae = 99999
min_i, min_j, min_k = 0, 0, 0
for i in tqdm(range(1, 26)):
    for j in range(1, 26):
        for k in range(2, 14, 2):
            RF_regr = RandomForestRegressor(max_depth=i, 
                                            n_estimators=j, 
                                            max_features=k,
                                            random_state=45
                                                )
            RF_regr.fit(X_train, np.ravel(Y_train))
            RF_Y_pred = RF_regr.predict(X_test)

            mae = mean_absolute_error(Y_test, RF_Y_pred)
            if (min_mae > mae):
                min_mae = mae
                min_i = i
                min_j = j
                min_k = k
            
print(min_mae, min_i, min_j, min_k)

100%|██████████| 25/25 [01:38<00:00,  3.92s/it]

2.049259538289614 9 7 2


## Decision Tree

In [14]:
min_mae = 99999

min_i, min_j, min_k  = 0, 0, 0

for i in tqdm(range(1, 26)):
    for j in range(1, 13):
        for k in range(2, 45, 1):
            
            DT_regr = DecisionTreeRegressor(max_depth=i,
                                max_features=j,
                                random_state=k)
            
            DT_regr.fit(X_train, Y_train)

            DT_Y_pred = DT_regr.predict(X_test)

            mae = mean_absolute_error(Y_test, DT_Y_pred)
            
            if (min_mae > mae):
                min_mae = mae
                min_i = i
                min_j = j
                min_k = k
            
print(min_mae, min_i, min_j, min_k)

100%|██████████| 25/25 [01:00<00:00,  2.43s/it]

0.8534917355371903 7 3 42


## Extra Trees

In [15]:
min_mae = 99999
min_i, min_j, min_k = 0, 0, 0
for i in tqdm(range(1, 30)):
    for j in range(1, 13):
        for k in range(2, 40, 1):
            ET_regr = ExtraTreesRegressor(n_estimators=i, 
                                            max_features=j,
                                            random_state=k
                                                )
            ET_regr.fit(X_train, np.ravel(Y_train))
            ET_Y_pred = ET_regr.predict(X_test)

            mae = mean_absolute_error(Y_test, ET_Y_pred)
            if (min_mae > mae):
                min_mae = mae
                min_i = i
                min_j = j
                min_k = k
            
print(min_mae, min_i, min_j, min_k)

100%|██████████| 29/29 [04:16<00:00,  8.84s/it]

0.9468181818181819 2 3 26


## AdaBoost

In [16]:
min_mae = 99999
min_i, min_j = 0, 0
for i in tqdm(range(1, 30)):
    for j in range(1, 30):
        AB_regr = AdaBoostRegressor(n_estimators=i, loss='exponential', learning_rate=j)
        AB_regr.fit(X_train, np.ravel(Y_train))
        AB_Y_pred = AB_regr.predict(X_test)
        AB_mae = mean_absolute_error(Y_test, AB_Y_pred)
        if (min_mae > mae):
                min_mae = mae
                min_i = i
                min_j = j
print(min_mae, min_i, min_j)

100%|██████████| 29/29 [00:18<00:00,  1.58it/s]

3.084858934169279 1 1


## Gradient Boosting

In [17]:
'''
min_mae = 999
min_i, min_j, min_k, min_l = 0, 0, 0.0, 0
for i in tqdm(range(300, 450, 10)):
    for j in range(2, 30, 2):
        for k in np.arange(0.06, 0.22, 0.02):
            for l in range(2, 22, 2):
                GB_regr = GradientBoostingRegressor(n_estimators=i, max_depth=j, learning_rate=k, random_state=l)
                GB_regr.fit(X_train, np.ravel(Y_train))
                GB_Y_pred = GB_regr.predict(X_test)

                mae = mean_absolute_error(Y_test, GB_Y_pred)
                if (min_mae > mae):
                    min_mae = mae
                    min_i = i
                    min_j = j
                    min_k = k
                    min_l = l

print(min_mae, min_i, min_j, min_k, min_l)
'''

'\nmin_mae = 999\nmin_i, min_j, min_k, min_l = 0, 0, 0.0, 0\nfor i in tqdm(range(300, 450, 10)):\n    for j in range(2, 30, 2):\n        for k in np.arange(0.06, 0.22, 0.02):\n            for l in range(2, 22, 2):\n                GB_regr = GradientBoostingRegressor(n_estimators=i, max_depth=j, learning_rate=k, random_state=l)\n                GB_regr.fit(X_train, np.ravel(Y_train))\n                GB_Y_pred = GB_regr.predict(X_test)\n\n                mae = mean_absolute_error(Y_test, GB_Y_pred)\n                if (min_mae > mae):\n                    min_mae = mae\n                    min_i = i\n                    min_j = j\n                    min_k = k\n                    min_l = l\n\nprint(min_mae, min_i, min_j, min_k, min_l)\n'

## Others

In [18]:
REGRESSIONS = {
    "K-nn": KNeighborsRegressor(),                          
    "Ridge": RidgeCV(),
    "Lasso": Lasso(),
    "ElasticNet": ElasticNet(random_state=0),
                }
# mean absolute error is used to evaluate the performance of all regressions.


for name, reg in REGRESSIONS.items():     
    reg.fit(X_train, Y_train)                 
    Y_pred = pd.DataFrame(reg.predict(X_test))
    
    print(name)
    
    mae = mean_absolute_error(Y_test, Y_pred)
    
    print(' MAE for Abs is ', mae, '\n')

K-nn
 MAE for Abs is  3.9115454545454544 

Ridge
 MAE for Abs is  5.895414558187242 

Lasso
 MAE for Abs is  6.304252234159574 

ElasticNet
 MAE for Abs is  6.192253873982705 



## Saving the best model

In [19]:
DT_regr = DecisionTreeRegressor(max_depth=7,
                                max_features=3,
                                random_state=42)
            
DT_regr.fit(X_train, Y_train)
DT_Y_pred = DT_regr.predict(X_test)

joblib.dump(DT_regr, "./model_aug_size_DecisionTree.joblib")

['./model_aug_size_DecisionTree.joblib']